<a href="https://colab.research.google.com/github/mirpouya/TensorFlow-Tutorial/blob/main/Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
#from tensorflow.keras.layers import InputLayer, Conv2D, Dense, BatchNormalization, MaxPool2D, Flatten
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.losses import BinaryCrossentropy
#from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.models import Model

In [5]:
dataset, dataset_info = tfds.load("malaria", with_info=True, as_supervised=True, shuffle_files=True, split=["train"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteS9ZTYJ/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

In [6]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):

  SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * SIZE))
  val_dataset = dataset.skip(int(TRAIN_RATIO * SIZE)).take(int(VAL_RATIO * SIZE))
  test_dataset = dataset.skip(int((TRAIN_RATIO + VAL_RATIO) * SIZE))

  return train_dataset, val_dataset, test_dataset

In [7]:
train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

In [8]:
IM_SIZE = 224

def resize_rescale(image, label, image_size = IM_SIZE):
  return tf.image.resize(image, (image_size, image_size)) / 255.0, label

In [9]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [10]:
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [12]:
lenet_loaded_model = tf.keras.models.load_model("lenet.hdf5")
lenet_loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_6 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_7 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

In [13]:
lenet_loaded_model.evaluate(test_dataset)

87/87 [==============================] - 38s 333ms/step - loss: 0.1732 - accuracy: 0.9434


[0.173234760761261, 0.9433962106704712]